In [16]:
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)
pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [17]:
bnf = pd.read_csv('BNF_mtrend.csv')
nifty = pd.read_csv('NIFTY_mtrend.csv')
# fnf = pd.read_csv('fnf_mtrend.csv')
midcp = pd.read_csv('MIDCP_mtrend.csv')
sensex = pd.read_csv('SENSEX_mtrend.csv')

bnf['Index'] = 'BANKNIFTY'
nifty['Index'] = 'NIFTY'
# fnf['Index'] = 'FINNIFTY'
midcp['Index'] = 'MIDCPNIFTY'
sensex['Index'] = 'SENSEX'

In [18]:
combined_df = pd.concat([bnf, nifty, midcp, sensex], ignore_index=True)
# combined_df.drop(columns=['Unnamed: 0'], inplace=True)

In [19]:
combined_df['Signal Generated At'] = pd.to_datetime(combined_df['Signal Generated At'])
combined_df = combined_df.sort_values(by='Signal Generated At')

In [20]:
combined_df = combined_df[combined_df['Trade Year'] >= 2019]

In [21]:
# bnf = bnf.dropna(subset=['Signal Generated At'])
# nifty = nifty.dropna(subset=['Signal Generated At'])
# fnf = fnf.dropna(subset=['Signal Generated At'])

In [22]:
combined_df['Allocation'] = 3750000
combined_df.loc[((combined_df['Trade Year'] >= 2019) & (combined_df['Trade Year'] <= 2021), 'Portfolio')] = 11250000
combined_df.loc[((combined_df['Trade Year'] >= 2022) & (combined_df['Trade Year'] <= 2025), 'Portfolio')] = 15000000
# combined_df.loc[((combined_df['Trade Year'] == 2022) & (combined_df['Trade Month'] >= 10)) | ((combined_df['Trade Year'] == 2023) & (combined_df['Trade Month'] <= 8)), 'Allocation'] = 6666666
# combined_df.loc[((combined_df['Trade Year'] >= 2023) & (combined_df['Trade Month'] >= 9)) | (combined_df['Trade Year'] >= 2024), 'Allocation'] = 5000000

In [23]:
# combined_df['Leverage'] = combined_df['Index'].map({
#     'BANKNIFTY': 5,
#     'FINNIFTY': 15
# }).fillna(8)  # Default leverage is 8 if neither BANKNIFTY nor FINNIFTY

# Continue with calculations
# combined_df['Qty'] = combined_df['Allocation'] * combined_df['Leverage'] / combined_df['Strike']
# combined_df['PnL'] = combined_df['After Costs'] * combined_df['Qty']
# combined_df['ROI%'] = combined_df['PnL'] * 100 / combined_df['Allocation']
# combined_df['ROI% PF'] = combined_df['PnL'] * 100 / combined_df['Portfolio']

In [24]:
combined_df.tail()

,Signal Generated At,Trade Type,Entry Time,Entry Price,Initial SL,Final SL,Exit Time,Exit Price,Points Captured,After Costs,PnL,Remarks,Qty,Leverage,ROI%,Trade Year,Trade Month,Index,Allocation,Portfolio
1414,2025-05-27 10:15:00,LONG,2025-05-27 11:15:00,24960.2000,24751.3500,24751.3500,2025-05-27 13:15:00,24751.3500,-208.8500,-218.7923,-246141.3488,Initial SL Hit,1125,7.5000,-6.5638,2025,5,NIFTY,3750000,15000000.0000
1673,2025-05-27 10:15:00,LONG,2025-05-27 11:15:00,12623.9500,12560.8500,12636.0300,2025-05-28 14:15:00,12599.8500,-24.1000,-36.7119,-48459.7080,TSL Hit,1320,4.5000,-1.2923,2025,5,MIDCPNIFTY,3750000,15000000.0000
1415,2025-05-29 11:15:00,LONG,2025-05-29 12:15:00,24748.3000,24708.6500,24708.6500,2025-05-29 12:15:00,24708.6500,-39.6500,-49.5414,-55734.0637,Initial SL Hit,1125,7.5000,-1.4862,2025,5,NIFTY,3750000,15000000.0000
1416,2025-05-29 13:15:00,LONG,2025-05-29 14:15:00,24773.7000,24717.8000,24775.0094,2025-05-30 10:15:00,24717.8000,-55.9000,-65.7983,-74023.0875,Initial SL hit,1125,7.5000,-1.9739,2025,5,NIFTY,3750000,15000000.0000
2318,2025-05-29 14:15:00,LONG,2025-05-29 15:15:00,81677.0900,81225.4000,81417.0993,2025-05-30 10:15:00,81384.3100,-292.7800,-309.0861,-105089.2876,TSL Hit,340,7.5000,-2.8024,2025,5,SENSEX,3750000,15000000.0000


In [25]:
combined_df['ROI% PF'] = combined_df['PnL'] * 100 / combined_df['Portfolio']

In [26]:
combined_df

,Signal Generated At,Trade Type,Entry Time,Entry Price,Initial SL,Final SL,Exit Time,Exit Price,Points Captured,After Costs,PnL,Remarks,Qty,Leverage,ROI%,Trade Year,Trade Month,Index,Allocation,Portfolio,ROI% PF
903,2019-01-01 13:15:00,LONG,2019-01-01 14:15:00,10830.3500,10807.6500,10857.5219,2019-01-02 12:15:00,10807.6500,-22.7000,-27.0276,-70947.4500,Initial SL hit,2625,7.5000,-1.8919,2019,1,NIFTY,3750000,11250000.0000,-0.6306
1741,2019-01-01 14:15:00,LONG,2019-01-01 15:15:00,36266.4900,35930.3400,36070.5543,2019-01-02 12:15:00,35930.3400,-336.1500,-343.3697,-267828.3527,Initial SL hit,780,7.5000,-7.1421,2019,1,SENSEX,3750000,11250000.0000,-2.3807
201,2019-01-01 14:15:00,LONG,2019-01-01 15:15:00,27395.1000,27076.6000,27210.9967,2019-01-02 12:15:00,27210.4000,-184.7000,-190.1605,-154030.0455,TSL Hit,810,6.0000,-4.1075,2019,1,BANKNIFTY,3750000,11250000.0000,-1.3692
904,2019-01-02 13:15:00,LONG,2019-01-02 14:15:00,10809.3000,10735.8000,10735.8000,2019-01-03 11:15:00,10735.8000,-73.5000,-77.8090,-204248.6775,Initial SL Hit,2625,7.5000,-5.4466,2019,1,NIFTY,3750000,11250000.0000,-1.8155
1742,2019-01-02 13:15:00,LONG,2019-01-02 14:15:00,35948.6400,35734.1300,35734.1300,2019-01-03 11:15:00,35734.1300,-214.5100,-221.6783,-172909.0561,Initial SL Hit,780,7.5000,-4.6109,2019,1,SENSEX,3750000,11250000.0000,-1.5370
202,2019-01-03 12:15:00,LONG,2019-01-03 13:15:00,27087.8000,26977.7500,26977.7500,2019-01-03 13:15:00,26977.7500,-110.0500,-115.4566,-96983.5062,Initial SL Hit,840,6.0000,-2.5862,2019,1,BANKNIFTY,3750000,11250000.0000,-0.8621
1743,2019-01-03 12:15:00,LONG,2019-01-03 13:15:00,35713.9900,35626.9900,35626.9900,2019-01-03 13:15:00,35626.9900,-87.0000,-94.1341,-73424.5964,Initial SL Hit,780,7.5000,-1.9580,2019,1,SENSEX,3750000,11250000.0000,-0.6527
203,2019-01-04 09:15:00,LONG,2019-01-04 10:15:00,27085.1000,26927.1000,26927.1000,2019-01-04 10:15:00,26927.1000,-158.0000,-163.4012,-137257.0248,Initial SL Hit,840,6.0000,-3.6602,2019,1,BANKNIFTY,3750000,11250000.0000,-1.2201
204,2019-01-04 11:15:00,LONG,2019-01-04 12:15:00,27158.0000,27064.4000,27081.3267,2019-01-04 14:15:00,27064.4000,-93.6000,-99.0222,-83178.6816,Initial SL hit,840,6.0000,-2.2181,2019,1,BANKNIFTY,3750000,11250000.0000,-0.7394
1744,2019-01-07 09:15:00,LONG,2019-01-07 10:15:00,36030.1700,35895.8300,35895.8300,2019-01-07 13:15:00,35895.8300,-134.3400,-141.5326,-110395.4280,Initial SL hit,780,7.5000,-2.9439,2019,1,SENSEX,3750000,11250000.0000,-0.9813


In [27]:
def generate_stats(tb_expiry):
    stats_df8 = pd.DataFrame(
        index=range(2019, 2026),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2019, 2026):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI% PF"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI% PF"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI% PF"] > 0]["ROI% PF"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI% PF"] < 0]["ROI% PF"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI% PF"].cumsum() - year_trades["ROI% PF"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        # variation = f'{signal_ma} , {trailing_ma}, {time_of_day}'
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            # variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI% PF"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI% PF"] > 0]["ROI% PF"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI% PF"] < 0]["ROI% PF"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI% PF"].cumsum() - combined_df_sorted["ROI% PF"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    # overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        # overall_variation,
    ]
    return pd.DataFrame(stats_df8)

In [28]:
stats = generate_stats(combined_df)
stats

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2019,89.7328,243,30.8642,3.1806,-0.8858,-19.4862,4.6049
2020,220.1013,241,33.6100,5.0645,-1.1882,-27.3267,8.0544
2021,158.4880,257,36.1868,3.2658,-0.8856,-17.7349,8.9365
2022,89.9760,340,33.8235,2.2181,-0.7338,-25.6352,3.5099
2023,115.1276,318,37.1069,1.7968,-0.4845,-14.1370,8.1437
2024,50.5714,336,30.9524,1.8016,-0.5896,-17.7900,2.8427
2025,31.6059,131,25.9542,2.9081,-0.6935,-17.4772,1.8084
Overall,755.6031,1866.0000,33.2262,2.7513,-0.7626,-30.3408,24.9038


# Mtrend - BNF, MIDCP, NIFTY, SENSEX

In [29]:
combined_df["DD%"] = combined_df["ROI% PF"].cumsum() - combined_df["ROI% PF"].cumsum().cummax()

In [37]:
combined_df.tail(24)

,Signal Generated At,Trade Type,Entry Time,Entry Price,Initial SL,Final SL,Exit Time,Exit Price,Points Captured,After Costs,PnL,Remarks,Qty,Leverage,ROI%,Trade Year,Trade Month,Index,Allocation,Portfolio,ROI% PF,DD%,Time In Trade,Premium Cost,Final Points,Final ROI%
1671,2025-05-12 09:15:00,LONG,2025-05-12 10:15:00,12445.0500,12322.1500,12800.8367,2025-05-19 13:15:00,12769.5500,324.5000,311.8927,411698.3640,TSL Hit,1320,4.5000,10.9786,2025,5,MIDCPNIFTY,3750000,15000000.0000,2.5986,-1.4643,8,16.5934,295.2993,2.5986
1408,2025-05-12 10:15:00,LONG,2025-05-12 11:15:00,24723.5500,24669.1000,24669.1000,2025-05-13 10:15:00,24669.1000,-54.4500,-64.3285,-72369.5963,Initial SL hit,1125,7.5000,-1.9299,2025,5,NIFTY,3750000,15000000.0000,-0.5134,-1.9467,1,4.1206,-68.4491,-0.5134
2312,2025-05-12 10:15:00,LONG,2025-05-12 11:15:00,81763.7100,81580.3400,81580.3400,2025-05-13 09:15:00,81580.3400,-183.3700,-199.7044,-67899.4977,Initial SL hit,340,7.5000,-1.8107,2025,5,SENSEX,3750000,15000000.0000,-0.4836,-2.3994,1,13.6273,-213.3317,-0.4836
2313,2025-05-13 13:15:00,LONG,2025-05-13 14:15:00,81300.3800,81138.7800,81138.7800,2025-05-13 14:15:00,81138.7800,-161.6000,-177.8439,-60466.9314,Initial SL Hit,340,7.5000,-1.6125,2025,5,SENSEX,3750000,15000000.0000,-0.4338,-2.8025,1,13.5501,-191.3940,-0.4338
2314,2025-05-14 09:15:00,LONG,2025-05-14 10:15:00,81664.2000,81184.8900,81184.8900,2025-05-14 12:15:00,81184.8900,-479.3100,-495.5949,-168502.2691,Initial SL Hit,340,7.5000,-4.4934,2025,5,SENSEX,3750000,15000000.0000,-1.1542,-3.9258,1,13.6107,-509.2056,-1.1542
1409,2025-05-14 09:15:00,LONG,2025-05-14 10:15:00,24758.5500,24581.7500,24581.7500,2025-05-14 12:15:00,24581.7500,-176.8000,-186.6681,-210001.5675,Initial SL Hit,1125,7.5000,-5.6000,2025,5,NIFTY,3750000,15000000.0000,-1.4310,-5.3259,1,4.1264,-190.7945,-1.4310
2315,2025-05-14 13:15:00,LONG,2025-05-14 14:15:00,81298.9300,81080.5600,81080.5600,2025-05-14 14:15:00,81080.5600,-218.3700,-234.6079,-79766.7027,Initial SL Hit,340,7.5000,-2.1271,2025,5,SENSEX,3750000,15000000.0000,-0.5625,-5.8576,1,13.5498,-248.1578,-0.5625
1410,2025-05-14 13:15:00,LONG,2025-05-14 14:15:00,24660.9500,24596.7500,24596.7500,2025-05-14 14:15:00,24596.7500,-64.2000,-74.0515,-83307.9825,Initial SL Hit,1125,7.5000,-2.2215,2025,5,NIFTY,3750000,15000000.0000,-0.5862,-6.4130,1,4.1102,-78.1617,-0.5862
1411,2025-05-15 09:15:00,LONG,2025-05-15 10:15:00,24706.7500,24494.4500,24986.6688,2025-05-19 12:15:00,24980.5500,273.8000,263.8625,296845.3575,TSL Hit,1125,7.5000,7.9159,2025,5,NIFTY,3750000,15000000.0000,1.8246,-4.4341,5,20.5890,243.2736,1.8246
2316,2025-05-15 09:15:00,LONG,2025-05-15 10:15:00,81439.0400,80762.1600,82322.7786,2025-05-19 11:15:00,82215.0000,775.9600,759.5946,258262.1626,TSL Hit,340,7.5000,6.8870,2025,5,SENSEX,3750000,15000000.0000,1.5679,-2.7123,5,67.8659,691.7287,1.5679


In [31]:
#### combined_df.to_csv('MTrend_nbms_1x.csv', index=False)

# MTrend - BNF, NF, MIDCP, SENSEX

In [32]:
combined_df['Entry Time'] = pd.to_datetime(combined_df['Entry Time'])
combined_df['Exit Time'] = pd.to_datetime(combined_df['Exit Time'])
combined_df['Time In Trade'] = (combined_df['Exit Time'] - combined_df['Entry Time'] + dt.timedelta(days=1)).dt.days

In [33]:
combined_df['Premium Cost'] = (combined_df['Time In Trade'] * 0.5/30) * combined_df['Entry Price'] / 100

In [34]:
combined_df['Final Points'] = combined_df['After Costs'] - combined_df['Premium Cost']
combined_df['Final ROI%'] = combined_df['Final Points'] * combined_df['Qty'] * 100 / combined_df['Portfolio']

In [35]:
combined_df['PnL'].sum() , combined_df['Final Points'].sum()

(95778382.84743169, 90798.23989623661)

In [36]:
combined_df['ROI% PF'] = combined_df['Final ROI%']
stats2 = generate_stats(combined_df)
stats2

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2019,65.0863,243,29.2181,3.1635,-0.9274,-24.8772,2.6163
2020,194.2390,241,32.7801,4.9839,-1.2314,-29.4701,6.5910
2021,132.8231,257,35.0195,3.1964,-0.9272,-20.1349,6.5967
2022,67.4203,340,32.3529,2.1893,-0.7539,-28.0724,2.4017
2023,90.7940,318,33.9623,1.8129,-0.5000,-17.4278,5.2097
2024,28.2491,336,30.0595,1.7190,-0.6186,-18.9989,1.4869
2025,23.4237,131,25.1908,2.8641,-0.7254,-19.8092,1.1825
Overall,602.0354,1866.0000,31.7256,2.7209,-0.7918,-35.2348,17.0864


# MTrend -> 4 indices after Premium Costs

In [45]:
combined_df.to_csv('MTrend tb.csv', index=False)

In [46]:
# opt_buy = pd.read_csv('BNF_FNF_NF_MIDCP_Combined_MTrend_Opt_Buying.csv')
# combined_df = pd.concat([bnf, fnf, nifty, midcp], ignore_index=True)
# combined_df.drop(columns=['Unnamed: 0'], inplace=True)
# combined_df['Entry Time'] = pd.to_datetime(combined_df['Entry Time'])
# combined_df = combined_df.sort_values(by='Entry Time')

In [87]:
# combined_df

In [88]:
# combined_df = combined_df.drop(columns=['DATETIME', 'TODAYS OPEN', 'TODAYS CLOSE', 'POINTS CAPTURED TODAY', 'QTY', 'MTM', 'DD%'])
# combined_df['PnL'] = np.where(combined_df['Strategy'].isna(), 'OPT BUY', 'POSITIONAL')

In [89]:
# combined_df2.to_csv('Combined MTrend Shotgun.csv')

In [90]:
# combined_df2 = pd.concat([combined_df, opt_buy], ignore_index=True)
# combined_df2.drop(columns=['Unnamed: 0'], inplace=True)
# combined_df2['Entry Time'] = pd.to_datetime(combined_df2['Entry Time'])
# combined_df2 = combined_df2.sort_values(by='Entry Time')
# combined_df2 = combined_df2.drop(columns=['DATETIME', 'TODAYS OPEN', 'TODAYS CLOSE', 'POINTS CAPTURED TODAY', 'QTY', 'MTM', 'DD%'])
# combined_df2['Strategy'] = np.where(combined_df2['THEO PnL'].isna(), 'OPT BUY', 'POSITIONAL')

In [91]:
# combined_df2

In [92]:
# combined_df2 = combined_df2.drop(columns=['Drawdown', 'Cumulative ROI%', 'Running Max ROI%'])

In [93]:
# combined_df2['entry_date'] = np.where(combined_df2['Expiry'].isna(), combined_df2['Entry Time'].dt.date, combined_df2['Expiry'])

In [94]:
def generate_stats(tb):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
        ],
    )
    combined_df_sorted = tb
    
    # Iterate over each year
    for year in range(2017, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI% PF"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI% PF"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI% PF"] > 0]["ROI% PF"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI% PF"] < 0]["ROI% PF"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI% PF"].cumsum() - year_trades["ROI% PF"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI% PF"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI% PF"] > 0]["ROI% PF"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI% PF"] < 0]["ROI% PF"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI% PF"].cumsum() - combined_df_sorted["ROI% PF"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
    ]
    return stats_df8

In [95]:
stats = generate_stats(combined_df)
stats

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,68.5217,233,28.7554,2.4661,-0.5826,-15.8957,4.3107
2018,62.1041,184,29.3478,2.8728,-0.7156,-24.0541,2.5818
2019,92.7731,176,25.5682,3.9985,-0.6653,-19.7932,4.6871
2020,239.1324,166,32.5301,6.5561,-1.0259,-22.2313,10.7566
2021,95.6872,176,27.8409,4.2385,-0.8819,-18.4246,5.1935
2022,94.0058,226,24.7788,3.8474,-0.7144,-20.0542,4.6876
2023,109.1013,234,32.0513,2.4398,-0.4647,-18.4958,5.8987
2024,44.8799,179,26.8156,2.5621,-0.5962,-20.5050,2.1887
Overall,806.2054,1574.0000,28.4625,3.5344,-0.6902,-24.0541,33.5163


# MTrend NSE Indices New

In [96]:
combined_df.to_csv('MTrend NSE Indices New.csv', index=False)